# Bibliotecas

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/0 (e.g. pd.read_csv)
import re # for regex
import unicodedata
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn. feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn. naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score
import pickle

# Funções

## Normalizador

In [2]:
def norm(s):
    """
    Normaliza o texto colocando em caixa baixa e removendo acentuação/pontuação.

    Parameters
    ----------
    s : str
        Entrada de texto.

    Returns
    -------
    str
        Texto normalizado. Se `s` não for uma string, retorna inalterado.

    Examples
    --------
    >>> norm("Árvore")
    'arvore'

    >>> norm("Fomos à escola ontem, Fernando não estava lá.")
    'fomos a escola ontem fernando nao estava la'
    """
    if not isinstance(s, str):
        return s
        
    s = s.lower()
    s = s.replace("-", " ")
    
    s = ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
  
    return re.sub(r"[^A-Za-zÀ-ÿ ]+", "", s)

## Tokenizador

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

def rem_stopwords(text):
    stop_words = set(stopwords.words('portuguese'))
    words = word_tokenize(text)
    return [w for w in words if w not in stop_words]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rique\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rique\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\rique\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Radicalizador

In [4]:
def stem_txt(text):
    ss = SnowballStemmer('portuguese')
    return " ".join([ss.stem(w) for w in text])

# Dados

## Palavras classificadas

In [5]:
df_class_words = pd.read_csv('../data/sentilex_v1.0.0.csv', sep = ';')

## Texto

In [6]:
text_positive = """A experiência de hoje foi absolutamente fantástica! Desde o primeiro momento, a equipe demonstrou 
um profissionalismo e uma simpatia incríveis, superando todas as minhas expectativas. O produto não é apenas bom, é 
maravilhoso e funciona perfeitamente. A qualidade é evidente em cada detalhe. Estou genuinamente impressionado e 
muito satisfeito com o resultado. Com certeza recomendo a todos; foi uma escolha excelente e valeu cada centavo!"""

In [7]:
text_negative = """Minha experiência hoje foi absolutamente terrível. Desde o início, o atendimento demonstrou um 
descaso e uma falta de profissionalismo inacreditáveis, frustrando todas as minhas expectativas. O produto não é 
apenas ruim, é péssimo e falha constantemente. A baixa qualidade é evidente em cada detalhe. Estou genuinamente 
decepcionado e muito insatisfeito com o resultado. Com certeza não recomendo a ninguém; foi uma escolha horrível e 
um desperdício de dinheiro."""

In [8]:
text_neutral = """O evento começou pontualmente às 10h da manhã, conforme anunciado no cronograma. O palestrante 
principal falou por exatos 45 minutos, cobrindo os tópicos listados na descrição da palestra. Após a apresentação, 
os participantes foram direcionados ao salão adjacente para o intervalo. A organização informou que a próxima 
sessão começará em 15 minutos."""

In [9]:
ig_text = "É bem comum as pessoas terem gostos próprios."

In [10]:
dict_texts = {"texts":[text_neutral, text_positive, text_negative, ig_text]}

In [11]:
df_texts = pd.DataFrame(dict_texts) 
df_texts["texts"].astype("string")

0    O evento começou pontualmente às 10h da manhã,...
1    A experiência de hoje foi absolutamente fantás...
2    Minha experiência hoje foi absolutamente terrí...
3        É bem comum as pessoas terem gostos próprios.
Name: texts, dtype: string

# Normaliza os dados

In [12]:
df_texts.texts = df_texts.texts.apply(norm).astype("string")
df_class_words.word = df_class_words.word.apply(norm).astype("string")

# Tokeniza

In [13]:
df_texts.texts = df_texts.texts.apply(rem_stopwords)
df_class_words.word = df_class_words.word.apply(rem_stopwords)

# Radicaliza

In [14]:
df_texts.texts = df_texts.texts.apply(stem_txt)
df_class_words.word = df_class_words.word.apply(stem_txt)
df_class_words = df_class_words.dropna(subset=["polarity_n0"])

# Criação do modelo

In [16]:
x = np.array(df_class_words.iloc[:,0].values)
y = np.array(df_class_words.polarity_n0.values)
cv = CountVectorizer(max_features = 1000)
X = cv.fit_transform(df_class_words.word).toarray()
print("X.shape = ", X.shape)
print("y.shape = ", y.shape)

X.shape =  (7008, 1000)
y.shape =  (7008,)


In [17]:
trainx, testx, trainy, testy = train_test_split(X,y, test_size = 0.2, random_state = 9)
print("Train shapes : X = {}, y = {}".format(trainx.shape, trainy.shape))
print("Test shapes : X = {}, y = {}".format(testx.shape, testy.shape))

Train shapes : X = (5606, 1000), y = (5606,)
Test shapes : X = (1402, 1000), y = (1402,)


In [18]:
gnb, mnb, bnb = GaussianNB(), MultinomialNB(alpha = 1.0, fit_prior = True), BernoulliNB(alpha = 1.0, fit_prior = True)
gnb.fit(trainx, trainy)
mnb.fit(trainx, trainy)
bnb.fit(trainx, trainy)

,alpha,1.0
,force_alpha,True
,binarize,0.0
,fit_prior,True
,class_prior,None


In [19]:
ypg = gnb.predict(testx)
ypm = mnb.predict(testx)
ypb = bnb.predict(testx)

In [20]:
print("Gaussian = ", accuracy_score(testy, ypg))
print("Multinomial = ", accuracy_score(testy, ypm))
print("Bernoulli = ", accuracy_score(testy, ypb))

Gaussian =  0.3181169757489301
Multinomial =  0.6754636233951498
Bernoulli =  0.6726105563480742


In [21]:
pickle.dump(bnb, open('model1.pkl','wb'))

# Utilização do modelo

In [35]:
f1 = norm(text_negative)
f2 = rem_stopwords(f1)
f3 = stem_txt(f2)

bow, words = [], word_tokenize(f3)
for word in words:
    bow.append(words.count(word))

word_dict = cv.vocabulary_
pickle.dump(word_dict,open('bow.pkl','wb'))

inp = []
for i in word_dict:
    inp.append(f3.count(i[0]))
y_pred = mnb.predict(np.array(inp).reshape(1,1000))

y_pred

array([0.])